In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-43aacedb-05e5-4632-09f7-8454b76105ca)


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ELI5')

Install required packages and setup dataset w\ elasticsearch-7.7.1 already in chdir

In [ ]:
!pip install elasticsearch
!pip install faiss_gpu
!pip install nlp
!pip install transformers
!pip install apache_beam

In [ ]:
import nlp
eli5 = nlp.load_dataset('eli5')

In [ ]:
eli5['test_eli5'][12345]

In [ ]:
wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0')['train']

In [ ]:
print(len(wiki40b_snippets))

17553713


In [ ]:
print(wiki40b_snippets[0]['passage_text'])

Ági Szalóki Life She started singing as a toddler, considering Márta Sebestyén a role model. Her musical background is traditional folk music; she first won recognition for singing with Ökrös in a traditional folk style, and Besh o droM, a Balkan gypsy brass band. With these ensembles she toured around the world from the Montreal Jazz Festival, through Glastonbury Festival to the Théatre de la Ville in Paris, from New York to Beijing.
Since 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.
Until now, three of her six released albums


Sample around **150k** questions from this large dataset

In [ ]:
import torch
p_sample = 150000 / len(wiki40b_snippets)
masks = []
for i in range(len(wiki40b_snippets)):
  u = torch.rand(1).item()
  if (u < p_sample):
    masks.append(i)

In [ ]:
wiki40b_sub = torch.utils.data.Subset(wiki40b_snippets, masks)

In [ ]:
print(len(wiki40b_sub))

150285


In [ ]:
print(wiki40b_sub[0]['article_title'])
print(wiki40b_sub[1]['article_title'])
print(wiki40b_sub[2]['article_title'])
print(wiki40b_sub[500]['article_title'])
print(wiki40b_sub[-2]['article_title'])
print(wiki40b_sub[-1]['article_title'])


01 Gallery
1633 (novel)
1996 Football League Third Division play-off Final
Robarts Centre for Canadian Studies
Lillian Feickert
Trump Tower meeting


In [ ]:
print(wiki40b_snippets[-1]['article_title'])

Zychlin (Hasidic dynasty)


Leverage the pretrained tokenizer and model, we start encoding questions, answers and wikis and store them.

In [ ]:
from lfqa_utils import *
qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('cuda')
_ = qar_model.eval()

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
wiki40b_sub.num_rows = len(wiki40b_sub)

In [ ]:
if not os.path.isfile('wiki40b_150k_reps.dat'):
    make_qa_dense_index(
        qar_model, qar_tokenizer, wiki40b_sub, device='cuda:0',
        index_name='wiki40b_150k_reps.dat'
    )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 0.8069579601287842
50 42.33217978477478
100 91.12235140800476
150 150.89147639274597
200 213.60625791549683
250 278.7684471607208


In [ ]:
if not os.path.isfile('wiki40b_full_reps.dat'):
    make_qa_dense_index(
        qar_model, qar_tokenizer, wiki40b_snippets, device='cuda:0',
        index_name='wiki40b_full_reps.dat'
    )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 0.8059952259063721
50 40.22945737838745
100 79.62978672981262
150 119.14326071739197
200 158.60782766342163
250 198.113041639328
300 237.5256552696228
350 277.042005777359
400 316.4643669128418
450 355.9450647830963
500 395.2554597854614
550 434.6010413169861
600 473.91746068000793
650 513.2774932384491
700 552.8230791091919
750 592.1883733272552
800 631.5730066299438
850 671.1154100894928
900 710.4789941310883
950 749.8878996372223
1000 789.5154092311859
1050 829.0698997974396
1100 868.5392661094666
1150 907.9491546154022
1200 947.3421967029572
1250 986.7623460292816
1300 1026.2126297950745
1350 1065.8410556316376
1400 1105.2625958919525
1450 1144.696459054947
1500 1184.281000137329
1550 1223.76881980896
1600 1263.3502469062805
1650 1302.802945613861
1700 1342.4610030651093
1750 1381.862945318222
1800 1421.3493700027466
1850 1460.7836410999298
1900 1500.2021667957306
1950 1539.7707958221436
2000 1579.1695551872253
2050 1618.560914516449
2100 1658.2013466358185
2150 1697.710890054702

Get a even samller 30K documents.

In [ ]:
import torch
p_sample = 30000 / len(wiki40b_snippets)
tk_masks = []
for i in range(len(wiki40b_snippets)):
  u = torch.rand(1).item()
  if (u < p_sample):
    tk_masks.append(i)
wiki30k = torch.utils.data.Subset(wiki40b_snippets, tk_masks)


In [ ]:
wiki30k.num_rows = len(wiki30k)
if not os.path.isfile('wiki40b_30k_reps.dat'):
    make_qa_dense_index(
        qar_model, qar_tokenizer, wiki30k, device='cuda:0',
        index_name='wiki40b_30k_reps.dat'
    )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 2.5083694458007812
50 105.43386363983154


In [ ]:
print(wiki30k[0].keys())

dict_keys(['_id', 'article_title', 'end_character', 'end_paragraph', 'nlp_id', 'passage_text', 'section_title', 'start_character', 'start_paragraph', 'wiki_id'])


Save dataset, and ids used to generate this subdataset.

In [ ]:
import pickle as pkl
def save_data(data_path, out_path):
  with open(out_path, 'wb') as fout:
    pkl.dump(data_path, fout)
save_data(wiki30k, "wiki30k_dataset.pkl")

In [ ]:
save_data(wiki40b_sub, "wiki150k_dataset.pkl")

In [ ]:
test = pkl.load(open("wiki30k_dataset.pkl","rb"))

In [ ]:
print(len(test))
print(wiki30k.num_rows)

29727
29727


Save sampled id-array

In [ ]:
import numpy as np
def save_ids(path, arr):
  with open(path, "wb") as f:
    np.save(f, np.array(arr))

def load_ids(path):
  with open(path, "rb") as f:
    ids = np.load(f)
  return ids

In [ ]:
save_ids("ids_for_30k.npy", tk_masks)
save_ids("ids_for_150k.npy", masks)

Save 20 / 200 / 2000 questions and their embeddings.

In [ ]:
eli5['test_eli5'][12345]['title']

'Why does water heated to room temperature feel colder than the air around it?'

In [ ]:
len(eli5['test_eli5'])

24512

In [ ]:
def sampler(sample_num, total_size):
  p_sample = sample_num / total_size
  sample_id = []
  for i in range(total_size):
    u = torch.rand(1).item()
    if (u < p_sample):
      sample_id.append(i)
  return sample_id

In [ ]:
q_total_size = len(eli5['test_eli5'])
q_20_ids = sampler(20, q_total_size)
q_200_ids = sampler(200, q_total_size)
q_2000_ids = sampler(200, q_total_size)

In [ ]:
save_ids("ids_for_q_20.npy", q_20_ids)
save_ids("ids_for_q_200.npy", q_200_ids)
save_ids("ids_for_q_2000.npy", q_2000_ids)

In [ ]:
def save_ques(q_ids, path):  
  q = []
  for id in q_ids:
    q.append(eli5['test_eli5'][id]['title'])
  with open(path, "w") as f:
    f.write("\n".join(q))

In [ ]:
save_ques(q_20_ids, "question_20_text.tsv")

In [ ]:
save_ques(q_200_ids, "question_200_text.tsv")

In [ ]:
save_ques(q_2000_ids, "question_2000_text.tsv")

save questions representation

In [ ]:
def save_q_rep(path, q_ids):
  rep = np.zeros((len(q_ids), 128))
  idx = 0
  for id in q_ids:
    q = eli5['test_eli5'][id]['title']
    q_rep = embed_questions_for_retrieval([q], qar_tokenizer, qar_model)
    rep[idx] = q_rep
    idx += 1
  save_ids(path, rep)

In [ ]:
save_q_rep("q_20_reps.npy", q_20_ids)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
save_q_rep("q_200_reps.npy", q_200_ids)
save_q_rep("q_2000_reps.npy", q_2000_ids)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Retrieving wiki based on embeddings.

In [ ]:
faiss_res = faiss.StandardGpuResources()
wiki40b_passage_reps = np.memmap(
            'wiki40b_.dat',
            dtype='float32', mode='r',
            shape=(wiki40b_snippets.num_rows, 128)
)

wiki40b_index_flat = faiss.IndexFlatIP(128)
wiki40b_gpu_index = faiss.index_cpu_to_gpu(faiss_res, 1, wiki40b_index_flat)
wiki40b_gpu_index.add(wiki40b_passage_reps)

In [ ]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, device='cuda:1')

df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

In [ ]:
test_qa_list = [(exple['title'],
                ' '.join([a 
                          for i, (a, sc) in enumerate(zip(exple['answers']['text'], exple['answers']['score'])) \
                          if i == 0 or sc >= 3
                         ]))
                for exple in eli5['test_eli5']]

# We then compute word frequencies in answer text
answer_doc_freq = {}
for q, a in test_qa_list:
    for w in a.lower().split():
        answer_doc_freq[w] = answer_doc_freq.get(w, 0) + 1

# The IDF-recall function is then:
def da_idf_recall(doc, answer):
    d_words = dict([(w, True) for w in doc.lower().split()])
    a_words = answer.lower().split()   
    recall = sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words if w in d_words]) / \
                sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words])
    return recall

In [ ]:

def dense_ret_for_eval(question, n_ret):
    _, dense_res_list = query_qa_dense_index(
        question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, n_results=n_ret, device='cuda:1'
    )
    dense_doc = ' '.join([res['passage_text'] for res in dense_res_list])
    return dense_doc

def sparse_ret_for_eval(question, n_ret):
    _, sparse_res_list = query_es_index(
        question, es_client, index_name='wiki40b_snippets_100w', n_results=n_ret
    )
    sparse_doc = ' '.join([res['passage_text'] for res in sparse_res_list])
    return sparse_doc

dense_score = evaluate_retriever(test_qa_list, dense_ret_for_eval, da_idf_recall)
sparse_score = evaluate_retriever(test_qa_list, sparse_ret_for_eval, da_idf_recall)

df = pd.DataFrame({
    'IDF-Recall': [sparse_score['idf_recall'], dense_score['idf_recall']],
    'Time/Query': [sparse_score['retrieval_time'], dense_score['retrieval_time']],
}, index=[ 'Sparse', 'Dense'])
df.style.format({'IDF-Recall': "{:.4f}", 'Time/Query': "{:.4f}"})